# GIS 5571 Lab2
### Tzu Yu Ma
### October 31, 2023

### Part 1

#### Downloads .LAS files from MN DNR

In [1]:
import arcpy
import requests
import os
import subprocess

In [2]:
county_name = 'aitkin'
tile_number = '1942-32-08'

In [3]:
# Define the URL for the LiDAR data repository
download_url = f'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/county/{county_name}/laz/{tile_number}.laz'

# Function to download the LiDAR data
def download_laz_files(download_url, save_path):
    response = requests.get(download_url)
    if response.status_code == 200:
        with open(save_path, 'wb') as laz_file:
            laz_file.write(response.content)
            print(f'Downloaded {save_path}')
    else:
        print('Failed to download .LAZ file')

# Specify the local path where you want to save the downloaded LiDAR data
save_path = f'D:\\arc1\\lab2\\{county_name}_{tile_number}.laz'

# Call the download_laz_files function
download_laz_files(download_url, save_path)

Downloaded D:\arc1\lab2\aitkin_1942-32-08.laz


In [4]:
#download lastool
tool_url = 'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/tools/lastools/'
download_dir = 'D:\\arc1\\lab2\\'  # 修改为您要保存文件的实际目录

# tool in the file
tool_files = ['unziplas.bat', 'laszip.exe', 'las2las.exe']

# download all the file (not sure which one can unzip)
for tool_file in tool_files:
    download_url = tool_url + tool_file
    save_path = os.path.join(download_dir, tool_file)
    
    response = requests.get(download_url)
    
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f'Downloaded {tool_file} to {download_dir}')
    else:
        print(f'Failed to download {tool_file}')

print("Download completed.")

Downloaded unziplas.bat to D:\arc1\lab2\
Downloaded laszip.exe to D:\arc1\lab2\
Downloaded las2las.exe to D:\arc1\lab2\
Download completed.


In [5]:
# use the lastool
laszip_exe = 'D:/arc1/lab2/laszip.exe'  

input_laz_file = f'D:\\arc1\\lab2\\{county_name}_{tile_number}.laz'  
output_las_file = f'D:\\arc1\\lab2\\{county_name}_{tile_number}.las'  

# use subprocess to excute the tool
laszip_command = f'"{laszip_exe}" -i "{input_laz_file}" -o "{output_las_file}"'
subprocess.run(laszip_command, shell=True)

# check
if os.path.exists(output_las_file):
    print(f'Successfully converted {input_laz_file} to {output_las_file}')
else:
    print('Conversion failed. Check the laszip.exe path and file paths.')


Successfully converted D:\arc1\lab2\aitkin_1942-32-08.laz to D:\arc1\lab2\aitkin_1942-32-08.las


#### Converts the .LAS file into both a DEM and a TIN


In [6]:
las_file = f'D:\\arc1\\lab2\\{county_name}_{tile_number}.las'
dem_file = f'D:\\arc1\\lab2\\{county_name}_{tile_number}.tif'
tin_file = f'D:\\arc1\\lab2\\{county_name}_{tile_number}.tin'
sr = 'EPSG:4326'

# create las dataset
las_dataset = arcpy.management.CreateLasDataset(las_file, 'las_dataset', '')

# convert to TIN
arcpy.ddd.LasDatasetToTin(las_dataset, tin_file, 'WINDOW_SIZE', 0.1, '', '', '')

# concert to DEM
arcpy.conversion.LasDatasetToRaster(las_dataset, dem_file)

<Result 'D:\\arc1\\lab2\\aitkin_1942-32-08.tif'>

#### Exports PDFs of the DEM and TIN with correct visualization

In [4]:
# Set APRX and Layout
aprx = arcpy.mp.ArcGISProject('CURRENT')
layout_name = 'TIN'
tin_lyt = aprx.listLayouts(layout_name)[0]

# Export Layout to PDF
tin_lyt.exportToPDF(f'D:\\arc1\\lab2\\{county_name}_{tile_number}_tin.pdf', resolution = 300)

'D:\\arc1\\lab2\\aitkin_1942-32-08_tin.pdf'

In [8]:
# Set APRX and Layout
aprx = arcpy.mp.ArcGISProject('CURRENT')
dem_lyt = aprx.listLayouts('DEM')[0]

# Export Layout to PDF
dem_lyt.exportToPDF(f'D:\\arc1\\lab2\\{county_name}_{tile_number}_dem.pdf', resolution = 300)

'D:\\arc1\\lab2\\aitkin_1942-32-08_dem.pdf'

In [9]:
# print out the layout name I have in this aprx
aprx = arcpy.mp.ArcGISProject("CURRENT")

layout_names = [layout.name for layout in aprx.listLayouts()]
#print(layout_names)

### Do 2D & 3D visualization

In [ ]:
# use Global Scence to convert

### ETL for PRISM

In [11]:
from zipfile import ZipFile

#### download PRISM data

In [12]:
# create new folder
base_path = 'D:\\arc1\\lab2'
folder_name = 'data'

folder_path = os.path.join(base_path, folder_name)
os.mkdir(folder_path)
print(f"new folder '{folder_name}' successfully created.")

parentfolder_path = 'D:\\arc1\\lab2\\data'
prism_folder_name = 'PRISM'

prism_folder_path = os.path.join(parentfolder_path, prism_folder_name)
os.mkdir(prism_folder_path)
print(f"new folder '{prism_folder_name}' successfully created.")

new folder 'data' successfully created.
new folder 'PRISM' successfully created.


In [13]:
prism_url = 'https://ftp.prism.oregonstate.edu/normals_4km/ppt/PRISM_ppt_30yr_normal_4kmM4_all_bil.zip'
save_path = 'D:\\arc1\\lab2\\data\\PRISM'

prism_response = requests.get(prism_url)

if prism_response.status_code == 200:
    file_path = os.path.join(save_path, 'PRISM_ppt_30yr_normal_4kmM4_all_bil.zip')

    with open(file_path, 'wb') as f:
        f.write(prism_response.content)
    
    print(f'Downloaded PRISM data to {file_path}')
else:
    print('Failed to download')

print("Download completed.")


Downloaded PRISM data to D:\arc1\lab2\data\PRISM\PRISM_ppt_30yr_normal_4kmM4_all_bil.zip
Download completed.


In [14]:
# unzipped 
zip_file_path = 'D:\\arc1\\lab2\\data\\PRISM\\PRISM_ppt_30yr_normal_4kmM4_all_bil.zip'

subdirectory_name = 'PRISM_ppt_30yr_normal_4kmM4_all_bil'  
unzip_dir = os.path.join('D:\\arc1\\lab2\\data\\PRISM', subdirectory_name)

os.makedirs(unzip_dir, exist_ok=True) # make sure the PRISM folder is there 

with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

print(f'Unzipped PRISM data to {unzip_dir}')

Unzipped PRISM data to D:\arc1\lab2\data\PRISM\PRISM_ppt_30yr_normal_4kmM4_all_bil


#### Converting to Spacetime Cube

In [2]:
# create gdb
gdb_path = r'D:\arc1\lab2\data\PRISM\PRISM.gdb'

arcpy.management.CreateFileGDB(r'D:\arc1\lab2\data\PRISM', 'PRISM.gdb')

print(f'created: {gdb_path}')

created: D:\arc1\lab2\data\PRISM\PRISM.gdb


In [3]:
# Create Empty Mosaic
sr = arcpy.SpatialReference(4326)  
empty_mosaic = arcpy.management.CreateMosaicDataset(r'D:\arc1\lab2\data\PRISM\PRISM.gdb','prism_mosaic',sr)

In [4]:
#  Add Rasters to Mosaic Dataset
mosaic = arcpy.management.AddRastersToMosaicDataset(empty_mosaic, 'Raster Dataset', r'D:\arc1\lab2\data\PRISM\PRISM_ppt_30yr_normal_4kmM4_all_bil')

In [5]:
# Create and populate a Variable field in the mosaic Footprints table
arcpy.management.CalculateField(r'prism_mosaic\Footprint', 'Variable', '"Text"', "PYTHON3")


<Result 'prism_mosaic\\Footprint'>

In [6]:
# Create and populate a Timestamp field in the mosaic Footprints table
arcpy.management.CalculateField(r'prism_mosaic\Footprint', 'Timestamp', "DateAdd(Date(2023, 0, 1), $feature.OBJECTID-1, 'month')", 'ARCADE','', 'DATE')

<Result 'prism_mosaic\\Footprint'>

In [7]:
# using the Build Multidimensional Info to enable the Time mosaic
arcpy.management.BuildMultidimensionalInfo('prism_mosaic', 'Variable', 'Timestamp')

<Result 'prism_mosaic'>

In [8]:
# using the Make Multidimensional Raster Layer tool Convert the time-enabled mosaic
ml = arcpy.md.MakeMultidimensionalRasterLayer('prism_mosaic', r'D:\arc1\lab2\data\PRISM\prism_ml', None)

In [9]:
#  Create Space Time Cube from Multidimensional Raster Layer 
stc = arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer(ml, r'D:\arc1\lab2\data\PRISM\prism_stc.nc', 'ZEROS')

#### Export an animation of the timeseries